<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [0]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training mountain pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training forest pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation mountain pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation forest pictures

In [0]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [5]:
print('Total training mountain images:', num_mountain_tr)
print('Total training forest images:', num_forest_tr)

print('Total validation mountain images:', num_mountain_val)
print('Total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

Total training mountain images: 254
Total training forest images: 270
Total validation mountain images: 125
Total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [0]:
batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [8]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [9]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


## Instatiate Model

In [10]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten (pooling layer)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)

predictions = Dense(1, activation='sigmoid', name='OutputLayer')(x)
model = Model(resnet.input, predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Using binary crossentropy due to being a 0 / 1 issue.

model.summary()  # Let's make sure everything looks correct!

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Fit Model

In [11]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 7s 210ms/step - loss: 0.9103 - accuracy: 0.4770 - val_loss: 0.7201 - val_accuracy: 0.3466
Epoch 2/50
32/32 [==============================] - 5s 160ms/step - loss: 0.7024 - accuracy: 0.5569 - val_loss: 0.7264 - val_accuracy: 0.3580
Epoch 3/50
32/32 [==============================] - 5s 161ms/step - loss: 0.6575 - accuracy: 0.5908 - val_loss: 0.6448 - val_accuracy: 0.6989
Epoch 4/50
32/32 [==============================] - 5s 160ms/step - loss: 0.6190 - accuracy: 0.6427 - val_loss: 0.5441 - val_accuracy: 0.7102
Epoch 5/50
32/32 [==============================] - 5s 161ms/step - loss: 0.5570 - accuracy: 0.7405 - val_loss: 0.5726 - val_accuracy: 0.7557
Epoch 6/50
32/32 [==============================] - 5s 162ms/step - loss: 0.5420 - accuracy: 0.7046 - val_loss: 0.4295 - val_accuracy: 0.8409
Epoch 7/50
32/32 [==============================] - 5s 162ms/step - loss: 0.4438 - accuracy: 0.8144 - val_loss: 0.4008 - val_accuracy: 0.8239
Epoch 

In [0]:
# Going to make predictions for each image here....

from tensorflow.keras.preprocessing import image
def process_img_path(img_path):
  """For loading images."""
  return image.load_img(img_path, target_size=(224, 224))

def img_predictions(img):
  """Takes a single image as input and makes a classification prediction."""
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]  # Top 3 predictions
  print(results)

In [13]:
for directory in os.listdir(train_dir)[:-1]:  # For both mountain and forest directories....
  print(f"Predictions for First Five Images in '{directory}':")
  print('-' * 120)
  for image_file in os.listdir(train_dir + '/' + directory)[:5]:  # First 5 files in that directory
    if image_file == '.ipynb_checkpoints':
      pass
    else:
      image_to_predict = f'{train_dir}/{directory}/{image_file}'  # This will be our image to predict.
      print(f'Top 3 Predictions For: {directory}/{image_file}')
      img_predictions(process_img_path(image_to_predict))
      print('-' * 120)
  print('=' * 120)

Predictions for First Five Images in 'mountain':
------------------------------------------------------------------------------------------------------------------------
Top 3 Predictions For: mountain/land180.jpg
[('n09193705', 'alp', 0.8629033), ('n09468604', 'valley', 0.03242456), ('n03792972', 'mountain_tent', 0.010283834)]
------------------------------------------------------------------------------------------------------------------------
Top 3 Predictions For: mountain/nat57.jpg
[('n09193705', 'alp', 0.6216985), ('n09468604', 'valley', 0.26169926), ('n09246464', 'cliff', 0.081158295)]
------------------------------------------------------------------------------------------------------------------------
Top 3 Predictions For: mountain/land198.jpg
[('n09193705', 'alp', 0.62811935), ('n09468604', 'valley', 0.36794987), ('n03792782', 'mountain_bike', 0.00088001863)]
-------------------------------------------------------------------------------------------------------------------

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
# Define the Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

CHNLS = 3

model = Sequential()

model.add(Conv2D(50, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, CHNLS), name='InputLayer'))
model.add(MaxPooling2D((2,2), name='PoolingLayer1'))
model.add(Conv2D(75, (3,3), activation='relu', name='ConvLayer2'))
model.add(MaxPooling2D((2,2),  name='PoolingLayer2'))
model.add(Flatten())
model.add(Dense(50, activation='relu', name='HiddenDense'))
model.add(Dense(1, activation='sigmoid', name='OutputLayer'))

In [15]:
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (Conv2D)          (None, 222, 222, 50)      1400      
_________________________________________________________________
PoolingLayer1 (MaxPooling2D) (None, 111, 111, 50)      0         
_________________________________________________________________
ConvLayer2 (Conv2D)          (None, 109, 109, 75)      33825     
_________________________________________________________________
PoolingLayer2 (MaxPooling2D) (None, 54, 54, 75)        0         
_________________________________________________________________
flatten (Flatten)            (None, 218700)            0         
_________________________________________________________________
HiddenDense (Dense)          (None, 50)                10935050  
_________________________________________________________________
OutputLayer (Dense)          (None, 1)                 5

In [16]:
# Fit Model

custom = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 3s 107ms/step - loss: 0.6590 - accuracy: 0.8423 - val_loss: 0.1843 - val_accuracy: 0.9318
Epoch 2/50
32/32 [==============================] - 3s 95ms/step - loss: 0.2314 - accuracy: 0.9022 - val_loss: 0.1726 - val_accuracy: 0.9375
Epoch 3/50
32/32 [==============================] - 3s 95ms/step - loss: 0.1528 - accuracy: 0.9242 - val_loss: 0.1570 - val_accuracy: 0.9261
Epoch 4/50
32/32 [==============================] - 3s 96ms/step - loss: 0.1237 - accuracy: 0.9551 - val_loss: 0.1685 - val_accuracy: 0.9489
Epoch 5/50
32/32 [==============================] - 3s 96ms/step - loss: 0.0667 - accuracy: 0.9701 - val_loss: 0.1788 - val_accuracy: 0.9261
Epoch 6/50
32/32 [==============================] - 3s 95ms/step - loss: 0.0806 - accuracy: 0.9681 - val_loss: 0.2296 - val_accuracy: 0.9148
Epoch 7/50
32/32 [==============================] - 3s 96ms/step - loss: 0.0551 - accuracy: 0.9741 - val_loss: 0.1322 - val_accuracy: 0.9545
Epoch 8/50
3

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [17]:
from keras.preprocessing.image import ImageDataGenerator

# Creating train_datagen
train_datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.25,
        height_shift_range=0.25,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')

# Creating val_datagen
val_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=1,
                                 width_shift_range=0.15,
                                 height_shift_range=0.2,
                                 zoom_range=.1,
                                 horizontal_flip=True,
                                 vertical_flip=True)

Using TensorFlow backend.


In [18]:
train_generator = train_datagen.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [19]:
val_generator = val_datagen.flow_from_directory(batch_size=batch_size,
                                                           directory=validation_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 195 images belonging to 2 classes.


In [0]:
gen_model = model.fit(
    train_generator,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 10s 324ms/step - loss: 1.0694 - accuracy: 0.6926 - val_loss: 0.7607 - val_accuracy: 0.5341
Epoch 2/50
32/32 [==============================] - 10s 328ms/step - loss: 0.4974 - accuracy: 0.7864 - val_loss: 1.2033 - val_accuracy: 0.5852
Epoch 3/50
32/32 [==============================] - 10s 324ms/step - loss: 0.3970 - accuracy: 0.8463 - val_loss: 0.9695 - val_accuracy: 0.6080
Epoch 4/50
32/32 [==============================] - 10s 323ms/step - loss: 0.3543 - accuracy: 0.8822 - val_loss: 0.7363 - val_accuracy: 0.7273
Epoch 5/50
32/32 [==============================] - 10s 325ms/step - loss: 0.3059 - accuracy: 0.8802 - val_loss: 0.6224 - val_accuracy: 0.7557
Epoch 6/50
32/32 [==============================] - 11s 329ms/step - loss: 0.3247 - accuracy: 0.8683 - val_loss: 0.6170 - val_accuracy: 0.7102
Epoch 7/50
32/32 [==============================] - 10s 323ms/step - loss: 0.3013 - accuracy: 0.8743 - val_loss: 0.5282 - val_accuracy: 0.7784

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language